<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Имеется база данных клиентов некоторой организации. Нужно защитить данные так, чтобы не потерять качество моделей машинного обучения. 

**Задача** - Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Убедиться в неизменности качества предсказаний модели.

**Описание данных**

- *Признаки:* пол, возраст и зарплата застрахованного, количество членов его семьи.
- *Целевой признак:* количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

Загрузим данные, проведем первичный анализ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(df.head(10))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


Пропуски в данных отсутствуют. Зададим названия столбцов латинскими буквами в нижнем регистре.

In [5]:
df.columns = ['sex', 'age', 'income', 'family_members', 'insurance_events']

In [6]:
print(df.duplicated().sum())

153


В данных присутствуют дубликаты. Возможно, люди имеют одинаковые персональные данные, и дублирующиеся строки представляют разных людей. Мы этого не знаем, потому что столбец с идентификационным номером клиента отсутствует. Поэтому удалим дубликаты.

In [7]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
sex                 4847 non-null int64
age                 4847 non-null float64
income              4847 non-null float64
family_members      4847 non-null int64
insurance_events    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 189.5 KB
None


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Метод преобразования данных:** Домножение признаков на обратимую матрицу.

**Обоснование:** 

Домножим признаки на обратимую матрицу P, предсказания примут вид:

$$
a_1 = (XP)w_1 
$$

Формула обучения примет вид:

$$
w_1 = ((XP)^T (XP))^{-1} (XP)^T y =
$$

$$
= ([P^T (X^T X)]P)^{-1} P^T X^T y =
$$

$$
= P^{-1} [P^T (X^T X)]^{-1} P^T X^T y =
$$

$$
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y =
$$

$$
= P^{-1} (X^T X)^{-1} E X^T y =
$$

$$
= P^{-1} w 
$$

Тогда предсказания будут равны:

$$
a_1 = XPw_1 = XPP^{-1}w = XEw = Xw = a
$$

Таким образом, при домножении признаков линейной регрессии на обратимую матрицу $ P $ параметры линейной регрессии домножаются на обратную ей матрицу $ P^{-1} $, а предсказания остаются неизменными. Следовательно и качество модели остается неизменным.

## Алгоритм преобразования

**Алгоритм**

Пусть матрица признаков линейной регрессии X имеет размеры n*m, тогда:

1. Генерируем случайную квадратную матрицу P размера m*m
2. Проверяем эту матрицу на обратимость. Если проверка не пройдена, то генерируем еще раз и снова проверяем до тех пор, пока не получим обратимую матрицу
3. Домножаем матрицу признаков X на обратимую матрицу P. 

**Обоснование**

1. Матрицу генерируем функцией numpy.random.normal(), таким образом вероятность получить необратимую матрицу стремится к нулю.
3. Качество линейной регрессии не поменяется, т.к. не изменятся ее предсказания, доказано в параграфе **2 Умножение матриц**

## Проверка алгоритма

In [8]:
features = df.drop('insurance_events', axis=1)
target = df['insurance_events']

In [9]:
P_size = features.shape[1]
P = np.random.normal(50, 25, (P_size, P_size))
print(P)

[[26.93239928 34.99803523 21.95852588 41.10781456]
 [ 7.62391746 28.6604132  42.0946797  47.65036983]
 [45.82758592 40.92287947 16.33086314 31.02317408]
 [51.62152566 35.07797942 44.21242749 44.69919055]]


In [10]:
P_inv = np.linalg.inv(P)
print(P_inv)

[[ 0.03084281 -0.03997656 -0.02615613  0.03240468]
 [-0.10792612  0.07115986  0.11564182 -0.05686373]
 [-0.10315173  0.05432928  0.04566247  0.00525596]
 [ 0.15110488 -0.06341329 -0.10570909  0.0243742 ]]


In [11]:
print(df.columns)

Index(['sex', 'age', 'income', 'family_members', 'insurance_events'], dtype='object')


In [12]:
def get_pred(features, target):
    features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=123)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_test)
    result = r2_score(target_test, predictions)
    
    return predictions, result

In [13]:
features_original = features.copy()
features_new = features.copy() @ P

In [14]:
pred_original, result_original = get_pred(features_original, target)
pred_new, result_new = get_pred(features_new, target)

print('R2 score до преобразования {:.4f}'.format(result_original))
print('R2 score после преобразования {:.4f}'.format(result_new))

R2 score до преобразования 0.3992
R2 score после преобразования 0.3992


## Вывод

В данной работе было доказано, что качество линейной регрессии не изменится от домножения матрицы признаков на обратимую матрицу. Основываясь на данном свойстве, был представлен алгоритм кодирования персональной информации клиентов в исходных данных. Данный алгоритм был применен, результат был проверен. Качество линейной регресии не изменилось.